Translator APP


In [ ]:
!pip install accelerate gTTS gradio transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr
from gtts import gTTS
import os

In [ ]:
device ="cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
language_model_name = "Qwen/Qwen2-1.5B-Instruct"

In [ ]:
language_model = AutoModelForCausalLM.from_pretrained(language_model_name,
                                                  torch_dtype="auto",
                                                  device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(language_model_name)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def process_input(input_text,action):
  # if action == "Translate to English":
  #   prompt = f"Translate this text to English:{input_text}",
  #   lang =" en"
  # elif action == "Translate to French":
  #   prompt = f"Translate this text to French: {input_text}"
  #   lang = 'fr'
  # elif action == "Translate to Spanish":
  #   prompt = f"Translate this text to Spanish: {input_text}"
  #   lang = 'es'
  # elif action == "Translate to Bangla":
  #   prompt = f"Translate this text to Bangla: {input_text}"
  #   lang = 'bangla'
  # else:
  #   prompt = input_text
  #   lang = 'en'
  if action == "Translate to English":
        prompt = f"Please translate the following text into English：{input_text}"
        lang = "en"
  elif action == "Translate to Chinese":
        prompt = f"Please translate the following text into Chinese：{input_text}"
        lang = "zh-cn"
  elif action == "Translate to Japanese":
        prompt = f"Please translate the following text into Japanese：{input_text}"
        lang = "ja"
  else:
        prompt = input_text
        lang = "en"

  messages = [
      {'role':"system","content": "You are a helpful assistant that can translates in different language"},
      {'role':"user","content": prompt}
  ]

  text = tokenizer.apply_chat_template(messages,tokenizer=False,add_generation_prompt=True)

  model_inputs = tokenizer([text],return_tensors="pt").to(device)

  generated_ids = language_model.generate(
      model_inputs.inputs_ids,
      max_new_tokens=512)

  generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

  output_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  return output_text, lang

In [ ]:
def text_to_speech(text,lang):
  tts = gTTS(text=text,lang=lang)
  tts.save("output.mp3")
  return "output.mp3"

In [ ]:
def handle_interaction(input_text,action):
  output_text,lang = process_input(input_text,action)
  audio_file_name = text_to_speech(output_text,lang)
  return output_text,audio_file_name

In [ ]:
action_options = ["Translate to English", "Translate to Chinese", "Translate to Japanese", "Chat"]

In [ ]:
#Gardio App
iface = gr.Interface(fn=handle_interaction,
                     inputs=[
                             gr.Textbox(label="Input Text"),
                             gr.Dropdown(label="Action",choices=action_options)
                     ],
                     outputs=[
                              gr.Textbox(label="Output Text"),
                              gr.Audio(label="Output Audio")
                     ],
                     title="Translation and Chat App",
                     description = "Translate and Chat using the Qwen 2 LLM with TTS features",
                     theme = "gradio/soft"
)

In [ ]:
 if __name__ == "__main__":
   iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://77ba136eb6fd5304b1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
